In [2]:
import torch

In [3]:
pretrained_dlv3 = torch.load('checkpoint/deeplabv3_r50-d8_512x512_4x4_320k_coco-stuff164k_20210709_155403-51b21115.pth')
pretrained_maskformer = torch.load('checkpoint/iter_160000.pth')

In [4]:
# matching
# pretrained_coco['state_dict'] ->  pretrained_maskformer['state_dict']
#                       18~329  ->  6~317
#                       backbone.layer

In [5]:
sd_dlv3 = pretrained_dlv3['state_dict']
sd_mf = pretrained_maskformer['state_dict']

In [9]:
common_sd = set(sd_dlv3.keys()).intersection( set(sd_mf.keys()) )
len(common_sd)

312

In [ ]:
freeze_indices = []
freeze_keys = []

parameters = []
for i, key in enumerate(pretrained_maskformer['state_dict'].keys()):
    if key in pretrained_dlv3['state_dict'].keys():
        pretrained_maskformer['state_dict'][key] = pretrained_dlv3['state_dict'][key]
    
        freeze_indices.append(i)
        freeze_keys(key)

In [ ]:
# 예시
layer_names = []
for idx, (name, param) in enumerate(finetune_net.named_parameters()):
    layer_names.append(name)
    # print(f'{idx}: {name}')

retrain_lr = 0.01
finetune_lr = 0.001

parameters = []
for idx, name in enumerate(layer_names):
    """
    60: fc.weight
    61: fc.bias
    """
    if idx>=0 and idx<=59: # conv layers
        lr = finetune_lr
    elif idx>=60: # fc layer
        lr = retrain_lr

    parameters += [{'params': [p for n, p in finetune_net.named_parameters() if n == name and p.requires_grad],
                    'lr':     lr}]
    
optimizer = optim.SGD(parameters, momentum=0.9)